# Imports

In [ ]:
# %load_ext nb_black
# %config Completer.use_jedi = False
# import nest_asyncio
# from IPython.core.interactiveshell import InteractiveShell

# nest_asyncio.apply()
# InteractiveShell.ast_node_interactivity = "all"

In [1]:
from lazyft import hyperopt, backtest, remote
from lazyft.quicktools.quick_tools import Config
from rich import console, pretty, traceback
from lazyft.command_parameters import HyperoptParameters, BacktestParameters
from lazyft.reports import get_backtest_repo, get_hyperopt_repo
from lazyft.models import Strategy
from lazyft.hyperopt.celery import CeleryRunner
from lazyft.strategy import StrategyTools
import pandas as pd
from lazyft.quicktools.quick_tools import QuickTools
c = console.Console(width=300, soft_wrap=True)
print = c.print
pretty.install(console=c)
traceback.install(console=c, show_locals=True)

<bound method InteractiveShell.excepthook of <ipykernel.zmqshell.ZMQInteractiveShell object at 0x7fc0a428dc70>>

# Configuration

In [2]:
# chosen_conf = Config("config_binance.com.json")
# chosen_conf = Config('config_binance.us.json')
private_conf = Config("binancecom.json")
us_conf = Config("config_binance.us.json")
kucoin = Config("config_3_100_unlimited_usdt.json")
# us_conf_no_blacklist = Config("config_binance_no_bl.us.json")
int_conf = Config("config_binance.com.json")
priv_us = Config("binanceus.json")
priv_int = Config("binancecom.json")

# chosen_conf = config_com
# general
n_coins = 50
starting_balance = 100
max_open_trades = 4
stake_amount = starting_balance / max_open_trades
interval = "5m"
inf_interval = "1h"
# config_new['timeframe'] = interval
timerange = "20210601-20210701"
timerange = None

# h_strategies = ["CombinedBinHClucAndMADV9", "BollingerBands3"]
# strategies = [
#     "TTF",
#     "BollingerBands2",
#     "BollingerBands3",
#     "NFI5MOHO_WIP",
#     "NotAnotherSMAOffsetStrategyModHO-yiXgyD",
# ]

# Data Downloading

In [ ]:
QuickTools.download_data(kucoin, '5m 1h', 10, verbose=True)

# Hyperopting

In [33]:
days = 45

# VRA/USDT HTR/USDT ARX/USDT
params = HyperoptParameters(
    epochs=750,
    strategies=[
        "MyEnsembleStrategy-15b05336-cd26-405f-bbc2-5e7f5f5b3e9a"
#         "NASOSv4",
#         "NASOSv5",
#         "NotAnotherSMAOffsetStrategyHOv3",
#         "MultiOffsetLamboV0-3d05e06d-d499-4957-a7cc-561bce1e0b2d",
#         "Diamond",
#         "CombinedBinHClucAndMADV9-60faed9e-7d81-4b55-9f86-32c131ed862b",
#         "SMAOG"
#         "Elliotv8",
#         "BigZ07"
#         Strategy(id='b40294bf-de3b-4f7d-a6f7-a412709e549c')
#         "Solipsis"
#     "RalliV1-73f72fcd-a501-4789-91c3-0f813715a02c",
#         "MacheteV8b-cdcc6e59-cd55-43d7-a0d0-149004e18b5a",
#         'Solipsis-c8df4b31-cb10-4c78-9bee-e7f65094bd0f'
               ],
    config_path=kucoin,
    #     secrets_config=str(priv_us),
    days=days,
#     timerange="20200701-20201231",
    spaces="sell",
    loss='ROIAndProfitHyperOptLoss',
#     loss='WinRatioAndProfitRatioLoss',
    interval="5m",
    inf_interval="1h 15m",
    min_trades=int(days*0.67*0.75),
    pairs=kucoin.whitelist,
#     pairs=['KSM/USDT'],
    #     pairs=pairlist.load_pairlist_from_id("52V2DPay"),
    starting_balance=starting_balance,
    max_open_trades=3,
    jobs=-2,
    download_data=False,
    tag="",
    seed=111111,
#     ensemble=ensemble
)
print("strategy:",params.strategies[0],"\ntimerange: ", params.timerange, '\nspaces:', params.spaces, '\nepochs:', params.epochs,'\nmin trades:', params.min_trades,'\ntag:', params.tag)
# QuickTools.refresh_pairlist(us_conf, 100)

strategy: MyEnsembleStrategy-15b05336-cd26-405f-bbc2-5e7f5f5b3e9a 
timerange:  20210813-20210912 
spaces: sell 
epochs: 750 
min trades: 22 
tag: 20210813-20210912,sell

In [35]:
h_commands = hyperopt.create_commands(params, verbose=False)
h_runner = hyperopt.HyperoptRunner(h_commands[0], autosave=False)
h_runner.execute(background=True)
# manager = hyperopt.HyperoptManager(h_commands)
# manager.execute()

2021-09-27 16:31:05.085 | INFO     | lazyft.util:set_params_file:40 - Copying id file 15b05336-cd26-405f-bbc2-5e7f5f5b3e9a.json to strategy folder
2021-09-27 16:31:05.086 | INFO     | lazyft.strategy:create_strategy_params_filepath:97 - Getting parameters file of MyEnsembleStrategy
2021-09-27 16:31:05.088 | INFO     | lazyft.hyperopt.runner:pre_execute:139 - Hyperopting MyEnsembleStrategy with id "15b05336-cd26-405f-bbc2-5e7f5f5b3e9a"
2021-09-27 16:31:05.104 | INFO     | lazyft.hyperopt.runner:execute:160 - Process ID: 127011


In [ ]:
print(manager.failed_runners)

In [24]:
id = CeleryRunner.get_running_hyperopt_id()
h_runner = CeleryRunner.load("c3555b12-1849-4738-8675-6160adbe9252")

In [ ]:
h_runner.get_results()

In [12]:
print(h_runner.epoch_text)

2021-09-19 21:01:22,255 - freqtrade.configuration.configuration - INFO - Using pairs ['ADA/USDT', 'XRP/USDT', 'LUNA/USDT', 'CIRUS/USDT', 'DOGE/USDT', 'AVAX/USDT', 'VET/USDT', 'VRA/USDT', 'DOT/USDT', 'KCS/USDT', 'QI/USDT', 'COTI/USDT', 'SOL/USDT', 'MASK/USDT', 'DAG/USDT', 'ICP/USDT', 'LABS/USDT', 'EOS/USDT', 'LINK/USDT', 'MATIC/USDT', 'XLM/USDT', 'LTC/USDT', 'XTZ/USDT', 'SAND/USDT', 'ZEC/USDT', 'XDC/USDT', 'QNT/USDT', 'HTR/USDT', 'TRX/USDT', 'THETA/USDT', 'ALGO/USDT', 'KSM/USDT', 'GRT/USDT', 'AAVE/USDT', 'CHR/USDT', 'FTM/USDT', 'XAVA/USDT', 'ERG/USDT', 'CAKE/USDT', 'SHA/USDT', 'MIR/USDT', 'ARX/USDT', 'DASH/USDT', 'WAXP/USDT', 'AI/USDT', 'ONE/USDT', 'ENJ/USDT', 'BTT/USDT', 'BCH/USDT', 'PDEX/USDT', 'NWC/USDT', 'EWT/USDT', 'FIL/USDT', 'IOST/USDT', 'XMR/USDT', 'ZIL/USDT', 'SXP/USDT', 'CHZ/USDT', 'MANA/USDT', 'SUSHI/USDT', 'GALAX/USDT', 'UNI/USDT', 'NEO/USDT', 'CQT/USDT']

In [63]:
h_runner.live_output()

Output()

In [30]:
_ = !freqtrade hyperopt-show --best
report = h_runner.generate_report()
h_runner.save()

2021-09-27 16:28:26.079 | INFO     | lazyft.strategy:create_strategy_params_filepath:97 - Getting parameters file of MyEnsembleStrategy


HyperoptReport(report_id='28dde3b1-3f2f-457c-aac4-6eb9226ddae8', param_id='15b05336-cd26-405f-bbc2-5e7f5f5b3e9a', strategy='MyEnsembleStrategy', exchange='kucoin', balance_info={'starting_balance': 100, 'stake_amount': 'unlimited', 'max_open_trades': 3}, date=datetime.datetime(2021, 9, 27, 16, 28, 26, 94813), tag='20210813-20210912,roi stoploss trailing', performance=HyperoptPerformance(start_date=datetime.datetime(2021, 8, 13, 0, 0), end_date=datetime.datetime(2021, 9, 12, 0, 0), trades=96, wins=88, losses=7, draws=1, avg_profits=4.44, med_profit=3.53, tot_profit=300.66828903, profit_percent=300.67, avg_duration='1:33:00', loss=-3.78273, seed=1111), params_file=PosixPath('/home/sage/ftworkdir/saved_params/15b05336-cd26-405f-bbc2-5e7f5f5b3e9a.json'), hyperopt_file=PosixPath('/home/sage/ftworkdir/user_data/hyperopt_results/strategy_MyEnsembleStrategy_2021-09-27_09-16-50.fthypt'), pairlist=['ADA/USDT', 'XRP/USDT', 'LUNA/USDT', 'CIRUS/USDT', 'DOGE/USDT', 'AVAX/USDT', 'VET/USDT', 'VRA/USDT', 'DOT/USDT', 'KCS/USDT', 'QI/USDT', 'COTI/USDT', 'SOL/USDT', 'MASK/USDT', 'DAG/USDT', 'LABS/USDT', 'EOS/USDT', 'MATIC/USDT', 'XLM/USDT', 'LTC/USDT', 'XTZ/USDT', 'ZEC/USDT', 'QNT/USDT', 'HTR/USDT', 'TRX/USDT', 'THETA/USDT', 'ALGO/USDT', 'KSM/USDT', 'GRT/USDT', 'AAVE/USDT', 'CHR/USDT', 'FTM/USDT', 'XAVA/USDT', 'CAKE/USDT', 'SHA/USDT', 'MIR/USDT', 'ARX/USDT', 'DASH/USDT', 'WAXP/USDT', 'AI/USDT', 'BTT/USDT', 'PDEX/USDT', 'NWC/USDT', 'EWT/USDT', 'FIL/USDT', 'IOST/USDT', 'XMR/USDT', 'ZIL/USDT', 'MANA/USDT', 'GALAX/USDT', 'UNI/USDT'])

In [33]:
h_runner.running

True

In [16]:
h_runner.stop()
# CeleryRunner.stop()

True

2021-09-27 09:15:39.963 | SUCCESS  | lazyft.hyperopt.runner:on_finished:215 - Finished successfully.
2021-09-27 09:15:40.230 | INFO     | lazyft.strategy:create_strategy_params_filepath:97 - Getting parameters file of MyEnsembleStrategy


In [26]:
print(h_runner.output)
# regex.EPOCH_LINE_REGEX.findall(h_runner.output)

2021-09-25 19:29:34,277 - freqtrade.configuration.configuration - INFO - Using config: /tmp/tmp6wcepdmb/config.json ...
2021-09-25 19:29:34,278 - freqtrade.loggers - INFO - Verbosity set to 0
2021-09-25 19:29:34,278 - freqtrade.configuration.configuration - INFO - Parameter -i/--timeframe detected ... Using timeframe: 5m ...
2021-09-25 19:29:34,278 - freqtrade.configuration.configuration - INFO - Parameter --max-open-trades detected, overriding max_open_trades to: 3 ...
2021-09-25 19:29:34,278 - freqtrade.configuration.configuration - INFO - Parameter --stake-amount detected, overriding stake_amount to: unlimited ...
2021-09-25 19:29:34,278 - freqtrade.configuration.configuration - INFO - Parameter --dry-run-wallet detected, overriding dry_run_wallet to: 100.0 ...
2021-09-25 19:29:34,278 - freqtrade.configuration.configuration - INFO - Parameter --timerange detected: 20210329-20210727 ...
2021-09-25 19:29:34,278 - freqtrade.configuration.configuration - INFO - Using user-data directory: /home/sage/ftworkdir/user_data ...
2021-09-25 19:29:34,298 - freqtrade.configuration.configuration - INFO - Using data directory: /home/sage/ftworkdir/user_data/data/kucoin ...
2021-09-25 19:29:34,299 - freqtrade.configuration.configuration - INFO - Overriding timeframe with Command line argument
2021-09-25 19:29:34,299 - freqtrade.configuration.configuration - INFO - Parameter --epochs detected ... Will run Hyperopt with for 750 epochs ...
2021-09-25 19:29:34,299 - freqtrade.configuration.configuration - INFO - Parameter -s/--spaces detected: ['buy', 'roi']
2021-09-25 19:29:34,299 - freqtrade.configuration.configuration - INFO - Parameter --no-color detected ...
2021-09-25 19:29:34,299 - freqtrade.configuration.configuration - INFO - Parameter -j/--job-workers detected: -2
2021-09-25 19:29:34,299 - freqtrade.configuration.configuration - INFO - Parameter --min-trades detected: 90
2021-09-25 19:29:34,299 - freqtrade.configuration.configuration - INFO - Using Hyperopt loss class name: ROIAndProfitHyperOptLoss
2021-09-25 19:29:34,299 - freqtrade.configuration.check_exchange - INFO - Checking exchange...
2021-09-25 19:29:34,303 - freqtrade.configuration.check_exchange - WARNING - Exchange "kucoin" is known to the the ccxt library, available for the bot, but not officially supported by the Freqtrade development team. It may work flawlessly (please report back) or have serious issues. Use it at your own discretion.
2021-09-25 19:29:34,303 - freqtrade.configuration.config_validation - INFO - Validating configuration ...
2021-09-25 19:29:34,305 - freqtrade.commands.optimize_commands - INFO - Starting freqtrade in Hyperopt mode
2021-09-25 19:29:34,305 - filelock - INFO - Lock 139713280311648 acquired on /home/sage/ftworkdir/user_data/hyperopt.lock
2021-09-25 19:29:34,306 - freqtrade.exchange.exchange - INFO - Instance is running with dry_run enabled
2021-09-25 19:29:34,306 - freqtrade.exchange.exchange - INFO - Using CCXT 1.56.30
2021-09-25 19:29:34,306 - freqtrade.exchange.exchange - INFO - Applying additional ccxt config: {'enableRateLimit': True}
2021-09-25 19:29:34,308 - freqtrade.exchange.exchange - INFO - Applying additional ccxt config: {'enableRateLimit': True, 'rateLimit': 200}
2021-09-25 19:29:34,311 - freqtrade.exchange.exchange - INFO - Using Exchange "KuCoin"
2021-09-25 19:29:35,701 - freqtrade.resolvers.exchange_resolver - INFO - Using resolved exchange 'Kucoin'...
2021-09-25 19:29:35,743 - freqtrade.resolvers.iresolver - INFO - Using resolved strategy Solipsis from '/home/sage/ftworkdir/user_data/strategies/solipsis5.py'...
2021-09-25 19:29:35,743 - freqtrade.strategy.hyper - INFO - Found no parameter file.
2021-09-25 19:29:35,743 - freqtrade.strategy.hyper - INFO - No params for buy found, using default values.
2021-09-25 19:29:35,743 - freqtrade.strategy.hyper - INFO - Strategy Parameter(default): buy_base_ma_streak = 1
2021-09-25 19:29:35,743 - freqtrade.strategy.hyper - INFO - Strategy Parameter(default): buy_base_mp = 30
2021-09-25 

# Backtesting

## Config

In [47]:
b_strategies = [
    "MyEnsembleStrategy-dfd605a4-afb0-43f0-8a49-87bb67e8aeb2",
#         "NotAnotherSMAOffsetStrategyModHO-94db2236-0861-4d7b-b720-7ecd04999290",
#     "NASOSv4",
#     "NostalgiaForInfinityNext",
#     "NotAnotherSMAOffsetStrategyHOv3Mod-92435106-95f4-4e78-bc4e-3f7ec1509f36",
#     "MultiMA_TSL",
#     Strategy(id='3da94866-ccb5-4821-af61-b69d687b8d04')
#     "NotAnotherSMAOffsetStrategyHOv3Mod",
#     "ElliotV8_original_ichiv2",
#     "Elliotv8",
#     "NASOSv4",
#     "NASOSv5",
#     "DIV_v1",
#     "BurkoBB1Strategy"
#     "TAD",
#     "Diamond",
#     "ElliotV3",
#     "Elliotv8",
#     "BigZ07",
#     "CombinedBinHClucAndMADV9",
#     "MADisplaceV3",
#     "SMAOG",
#     "Combined_NFIv7_SMA",
#     "NotAnotherSMAOffsetStrategyModHO",
#     "NotAnotherSMAOffsetStrategyHOv3",
#     "NFI5MOHO_WIP",
#     "TDLA",
#     "NostalgiaForInfinityV7",
#     "NotAnotherSMAOffsetStrategy_UZI",
#     "NotAnotherSMAOffSetStrategy_V2",
#     Strategy(id='7d818358-d86d-4301-9282-8aa32ecf359a'),
#     Strategy(id='b40294bf-de3b-4f7d-a6f7-a412709e549c'),
#     Strategy(id='ca1f7c9b-e3d6-483b-b912-00688410f0ed')
#     Strategy(id='20d6ad94-c1c1-4a77-9737-f4e3414dd1fb'), # MultiMA_TSL
#     Strategy(id='3da94866-ccb5-4821-af61-b69d687b8d04'), # NASOSv4
#     Strategy(id='94db2236-0861-4d7b-b720-7ecd04999290'), # NotAnotherSMAOffsetStrategyHOv3Mod
#     "RalliV1",
# "InverseV2",
# "NotAnotherSMAOffsetStrategy_uzi3",
# "MacheteV8b",
# "MultiOffsetLamboV0-3d05e06d-d499-4957-a7cc-561bce1e0b2d",
#     "MultiMA_TSL-4c7ab7ee-0a09-426c-9db5-0d6617262432"
]
ensemble=[
#     "NotAnotherSMAOffsetStrategyModHO-94db2236-0861-4d7b-b720-7ecd04999290",
#     "NotAnotherSMAOffsetStrategyHOv3Mod-92435106-95f4-4e78-bc4e-3f7ec1509f36",
#     Strategy(id='73f72fcd-a501-4789-91c3-0f813715a02c'), # RalliV1
#     Strategy(id='8da3c71a-3e34-49ed-a1ca-70bffa25600a') # Nostalgia...Next
#     Strategy(id='59523bd9-6c59-4edd-b990-ac48b1c6dfb9'), # NotAnotherSMAOffsetStrategyHOv3
#     Strategy(id='b648ca47-5f5b-4339-815f-ff6321c399d3'), # BigZ07
#     Strategy(id='4c7ab7ee-0a09-426c-9db5-0d6617262432'), # multima_TSL
    Strategy(id='03558ab6-cdc0-4ffa-930e-ab87d6e04c9a'),  # Elliotv8
    Strategy(id='1d22c9d0-78c4-4daa-bdbc-ca590914a495'),  # BigZ04_TSL4
    Strategy(id='3da94866-ccb5-4821-af61-b69d687b8d04'),  # NASOV4
    Strategy(id='18d342e5-44d0-42a9-90aa-b5dddc72571f'),  # NASOv5,
         ]
b_timerange = "20210925-"
# b_timerange = None
b_days = 365
b_interval = "5m 1h 15m"
b_conf = kucoin
# b_conf = kucoin
b_params = BacktestParameters(
    strategies=b_strategies,
    interval="5m",
    inf_interval="1h",
    config_path=b_conf,
    days=b_days,
    pairs=b_conf.whitelist,
#     pairs=['VRA/USDT', 'HTR/USDT', 'KSM/USDT'],
    stake_amount="unlimited",
    starting_balance=1000,
    max_open_trades=3,
    timerange=b_timerange,
    download_data=False,
    tag='trailing-fix',
    ensemble=ensemble
)
print(len(b_params.pairs), 'pairs')
print('timeframe:', b_params.timerange)
print('ensemble:', b_params.ensemble)

2021-09-28 16:12:30.641 | INFO     | lazyft.util:set_params_file:40 - Copying id file 03558ab6-cdc0-4ffa-930e-ab87d6e04c9a.json to strategy folder
2021-09-28 16:12:30.642 | INFO     | lazyft.strategy:create_strategy_params_filepath:97 - Getting parameters file of Elliotv8
2021-09-28 16:12:30.643 | INFO     | lazyft.util:set_params_file:40 - Copying id file 1d22c9d0-78c4-4daa-bdbc-ca590914a495.json to strategy folder
2021-09-28 16:12:30.644 | INFO     | lazyft.strategy:create_strategy_params_filepath:97 - Getting parameters file of BigZ04_TSL4
2021-09-28 16:12:30.645 | INFO     | lazyft.util:set_params_file:40 - Copying id file 3da94866-ccb5-4821-af61-b69d687b8d04.json to strategy folder
2021-09-28 16:12:30.646 | INFO     | lazyft.strategy:create_strategy_params_filepath:97 - Getting parameters file of NASOSv4
2021-09-28 16:12:30.648 | INFO     | lazyft.util:set_params_file:40 - Copying id file 18d342e5-44d0-42a9-90aa-b5dddc72571f.json to strategy folder
2021-09-28 16:12:30.648 | INFO  

52 pairs

timeframe: 20210925-

ensemble:
[Strategy(name='Elliotv8', id='03558ab6-cdc0-4ffa-930e-ab87d6e04c9a'), Strategy(name='BigZ04_TSL4', id='1d22c9d0-78c4-4daa-bdbc-ca590914a495'), Strategy(name='NASOSv4', id='3da94866-ccb5-4821-af61-b69d687b8d04'), Strategy(name='NASOSv5', id='18d342e5-44d0-42a9-90aa-b5dddc72571f')]

In [29]:
b_params.config.whitelist

[
    'ADA/USDT',
    'XRP/USDT',
    'LUNA/USDT',
    'CIRUS/USDT',
    'DOGE/USDT',
    'AVAX/USDT',
    'VET/USDT',
    'VRA/USDT',
    'DOT/USDT',
    'KCS/USDT',
    'QI/USDT',
    'COTI/USDT',
    'SOL/USDT',
    'MASK/USDT',
    'DAG/USDT',
    'ICP/USDT',
    'LABS/USDT',
    'EOS/USDT',
    'LINK/USDT',
    'MATIC/USDT',
    'XLM/USDT',
    'LTC/USDT',
    'XTZ/USDT',
    'SAND/USDT',
    'ZEC/USDT',
    'XDC/USDT',
    'QNT/USDT',
    'HTR/USDT',
    'TRX/USDT',
    'THETA/USDT',
    'ALGO/USDT',
    'KSM/USDT',
    'GRT/USDT',
    'AAVE/USDT',
    'CHR/USDT',
    'FTM/USDT',
    'XAVA/USDT',
    'ERG/USDT',
    'CAKE/USDT',
    'SHA/USDT',
    'MIR/USDT',
    'ARX/USDT',
    'DASH/USDT',
    'WAXP/USDT',
    'AI/USDT',
    'ONE/USDT',
    'ENJ/USDT',
    'BTT/USDT',
    'BCH/USDT',
    'PDEX/USDT',
    'NWC/USDT',
    'EWT/USDT',
    'FIL/USDT',
    'IOST/USDT',
    'XMR/USDT',
    'ZIL/USDT',
    'SXP/USDT',
    'CHZ/USDT',
    'MANA/USDT',
    'SUSHI/USDT',
    'GALAX/USDT',
    'UNI/USDT',
    'NEO/USDT',
    'CQT/USDT'
]

## Download

In [46]:
# QuickTools.download_data(us_conf, '30m', timerange='20200801-', pairs=['BTC/USD'], verbose=True)
# QuickTools.download_data(kucoin, '15m 4h', timerange=b_timerange, pairs=, verbose=False)
# QuickTools.download_data(kucoin, '15m 4h', timerange=b_timerange, pairs=b_conf.whitelist + ['BTC/USDT'], verbose=False)
# !freqtrade download-data -c {b_conf} -t 5m 15m 1h --timerange {b_timerange} -p {' '.join(b_conf.whitelist + ['BTC/USDT'])}
!freqtrade download-data -c {b_conf} -t 5m 15m 1h --timerange {b_timerange} -p 'BTC/USDT'

2021-09-28 16:12:04,037 - freqtrade.configuration.configuration - INFO - Using config: /home/sage/ftworkdir/configs/config_3_100_unlimited_usdt.json ...
2021-09-28 16:12:04,038 - freqtrade.loggers - INFO - Verbosity set to 0
2021-09-28 16:12:04,038 - freqtrade.configuration.configuration - INFO - Parameter --timerange detected: 20210925- ...
2021-09-28 16:12:04,038 - freqtrade.configuration.configuration - INFO - Using user-data directory: /home/sage/ftworkdir/user_data ...
2021-09-28 16:12:04,038 - freqtrade.configuration.configuration - INFO - Using data directory: /home/sage/ftworkdir/user_data/data/kucoin ...
2021-09-28 16:12:04,038 - freqtrade.configuration.configuration - INFO - Using pairs ['BTC/USDT']
2021-09-28 16:12:04,038 - freqtrade.configuration.configuration - INFO - timeframes --timeframes: ['5m', '15m', '1h']
2021-09-28 16:12:04,038 - freqtrade.configuration.check_exchange - INFO - Checking exchange...
2021-09-28 16:12:04,041 - freqtrade.configuration.check_exchange - W

## Single Run

In [5]:
# print(b_commands[0].command_string)
b_commands = backtest.create_commands(
    b_params,
    verbose=False,
)
b_runner = backtest.BacktestRunner(b_commands[0])
b_runner.execute()
# if b_runner.error:
#     print(b_runner.output)
# else:
#     b_runner.save()
#     get_backtest_repo().df().head(1)
print(b_runner.output)

2021-09-27 02:48:58.783 | INFO     | lazyft.strategy:create_strategy_params_filepath:97 - Getting parameters file of MyEnsembleStrategy3
2021-09-27 02:48:58.784 | INFO     | lazyft.util:remove_params_file:75 - Removing strategy params: /home/sage/ftworkdir/user_data/strategies/MyEnsembleStrategy3.json
2021-09-27 02:48:58.786 | INFO     | lazyft.backtest.runner:execute:104 - Backtesting MyEnsembleStrategy3 with params id "null" - 7094dce29f6329d0e805a14114fa9190
2021-09-27 03:03:26.842 | INFO     | lazyft.backtest.runner:on_finished:135 - Elapsed time: 868.06
2021-09-27 03:03:26.843 | SUCCESS  | lazyft.backtest.runner:on_finished:141 - MyEnsembleStrategy3 backtest completed successfully


2021-09-27 02:49:00,132 - freqtrade.configuration.configuration - INFO - Using config: /home/sage/ftworkdir/configs/config_3_100_unlimited_usdt.json ...
2021-09-27 02:49:00,133 - freqtrade.loggers - INFO - Verbosity set to 0
2021-09-27 02:49:00,133 - freqtrade.configuration.configuration - INFO - Parameter --max-open-trades detected, overriding max_open_trades to: 3 ...
2021-09-27 02:49:00,133 - freqtrade.configuration.configuration - INFO - Parameter --stake-amount detected, overriding stake_amount to: unlimited ...
2021-09-27 02:49:00,133 - freqtrade.configuration.configuration - INFO - Parameter --dry-run-wallet detected, overriding dry_run_wallet to: 1000.0 ...
2021-09-27 02:49:00,133 - freqtrade.configuration.configuration - INFO - Parameter --timerange detected: 20200820- ...
2021-09-27 02:49:00,133 - freqtrade.configuration.configuration - INFO - Using user-data directory: /home/sage/ftworkdir/user_data ...
2021-09-27 02:49:00,133 - freqtrade.configuration.configuration - INFO - Using data directory: /home/sage/ftworkdir/user_data/data/kucoin ...
2021-09-27 02:49:00,133 - freqtrade.configuration.check_exchange - INFO - Checking exchange...
2021-09-27 02:49:00,137 - freqtrade.configuration.check_exchange - WARNING - Exchange "kucoin" is known to the the ccxt library, available for the bot, but not officially supported by the Freqtrade development team. It may work flawlessly (please report back) or have serious issues. Use it at your own discretion.
2021-09-27 02:49:00,137 - freqtrade.configuration.config_validation - INFO - Validating configuration ...
2021-09-27 02:49:00,139 - freqtrade.commands.optimize_commands - INFO - Starting freqtrade in Backtesting mode
2021-09-27 02:49:00,139 - freqtrade.exchange.exchange - INFO - Instance is running with dry_run enabled
2021-09-27 02:49:00,139 - freqtrade.exchange.exchange - INFO - Using CCXT 1.56.86
2021-09-27 02:49:00,139 - freqtrade.exchange.exchange - INFO - Applying additional ccxt config: {'enableRateLimit': True}
2021-09-27 02:49:00,141 - freqtrade.exchange.exchange - INFO - Applying additional ccxt config: {'enableRateLimit': True, 'rateLimit': 200}
2021-09-27 02:49:00,144 - freqtrade.exchange.exchange - INFO - Using Exchange "KuCoin"
2021-09-27 02:49:01,501 - freqtrade.resolvers.exchange_resolver - INFO - Using resolved exchange 'Kucoin'...
2021-09-27 02:49:01,541 - freqtrade.resolvers.iresolver - INFO - Using resolved strategy MyEnsembleStrategy3 from '/home/sage/ftworkdir/user_data/strategies/MyEnsembleStrategy3.py'...
2021-09-27 02:49:01,542 - freqtrade.strategy.hyper - INFO - Found no parameter file.
2021-09-27 02:49:01,542 - freqtrade.strategy.hyper - INFO - No params for buy found, using default values.
2021-09-27 02:49:01,542 - freqtrade.strategy.hyper - INFO - Strategy Parameter(default): combo_sell_signal = 2
2021-09-27 02:49:01,542 - freqtrade.strategy.hyper - INFO - No params for sell found, using default values.
2021-09-27 02:49:01,542 - freqtrade.strategy.hyper - INFO - Strategy Parameter(default): pHSL = -0.195
2021-09-27 02:49:01,542 - freqtrade.strategy.hyper - INFO - Strategy Parameter(default): pPF_1 = 0.009
2021-09-27 02:49:01,542 - freqtrade.strategy.hyper - INFO - Strategy Parameter(default): pPF_2 = 0.063
2021-09-27 02:49:01,542 - freqtrade.strategy.hyper - INFO - Strategy Parameter(default): pSL_1 = 0.011
2021-09-27 02:49:01,542 - freqtrade.strategy.hyper - INFO - Strategy Parameter(default): pSL_2 = 0.062
2021-09-27 02:49:01,542 - freqtrade.strategy.hyper - INFO - No params for protection found, using default values.
2021-09-27 02:49:01,542 - MyEnsembleStrategy3 - INFO - Buy strategies: ['NotAnotherSMAOffsetStrategyModHO', 'Elliotv8', 'BigZ04_TSL4', 'NASOSv4']
2021-09-27 02:49:01,543 - freqtrade.resolvers.strategy_resolver - INFO - Override strategy 'timeframe' with value in config file: 5m.
2021-09-27 02:49:01,543 - freqtrade.resolvers.strategy_resolver - INFO - Override strategy 'order_types' with value in config file: {'buy': 'limit', 'sell'

In [10]:
print(b_runner.output)

## Multi-Run

In [52]:
b_commands = backtest.create_commands(
    b_params,
    verbose=False,
)
bm_runner = backtest.BacktestMultiRunner(b_commands)
bm_runner.execute()
if any(bm_runner.errors):
    print(bm_runner.errors)

bm_runner.save()
get_backtest_repo().head(len(bm_runner.reports)).df()

2021-09-28 16:19:28.927 | INFO     | lazyft.util:set_params_file:40 - Copying id file dfd605a4-afb0-43f0-8a49-87bb67e8aeb2.json to strategy folder
2021-09-28 16:19:28.929 | INFO     | lazyft.strategy:create_strategy_params_filepath:97 - Getting parameters file of MyEnsembleStrategy
2021-09-28 16:19:28.933 | INFO     | lazyft.backtest.runner:execute:104 - Backtesting MyEnsembleStrategy with params id "dfd605a4-afb0-43f0-8a49-87bb67e8aeb2" - 34184edcd779489c46ce7c188249ccbd
2021-09-28 16:20:00.312 | INFO     | lazyft.backtest.runner:on_finished:135 - Elapsed time: 31.38
2021-09-28 16:20:00.313 | SUCCESS  | lazyft.backtest.runner:on_finished:141 - MyEnsembleStrategy backtest completed successfully


,strategy,date,hyperopt_id,exchange,m_o_t,stake,balance,score,avg_profit_pct,total_profit_pct,roiloss,winratioloss,sharpe_loss,sortino,total_profit,trades,days,tag
0,MyEnsembleStrategy,09/28/21 16:20:00,dfd605a4-afb0-43f0-8a49-87bb67e8aeb2,kucoin,3,unlimited,1000,1.201,1.802,0.012,-1.605,-360.313,-19.987,20.000,11.920,2,3,trailing-fix


In [45]:
print(bm_runner.runners[0].output)

2021-09-26 21:29:12,519 - freqtrade.configuration.configuration - INFO - Using config: /home/sage/ftworkdir/configs/config_3_100_unlimited_usdt.json ...
2021-09-26 21:29:12,520 - freqtrade.loggers - INFO - Verbosity set to 0
2021-09-26 21:29:12,520 - freqtrade.configuration.configuration - INFO - Parameter --max-open-trades detected, overriding max_open_trades to: 3 ...
2021-09-26 21:29:12,520 - freqtrade.configuration.configuration - INFO - Parameter --stake-amount detected, overriding stake_amount to: unlimited ...
2021-09-26 21:29:12,520 - freqtrade.configuration.configuration - INFO - Parameter --dry-run-wallet detected, overriding dry_run_wallet to: 1000.0 ...
2021-09-26 21:29:12,520 - freqtrade.configuration.configuration - INFO - Parameter --timerange detected: 20200820- ...
2021-09-26 21:29:12,520 - freqtrade.configuration.configuration - INFO - Using user-data directory: /home/sage/ftworkdir/user_data ...
2021-09-26 21:29:12,520 - freqtrade.configuration.configuration - INFO - Using data directory: /home/sage/ftworkdir/user_data/data/kucoin ...
2021-09-26 21:29:12,520 - freqtrade.configuration.check_exchange - INFO - Checking exchange...
2021-09-26 21:29:12,524 - freqtrade.configuration.check_exchange - WARNING - Exchange "kucoin" is known to the the ccxt library, available for the bot, but not officially supported by the Freqtrade development team. It may work flawlessly (please report back) or have serious issues. Use it at your own discretion.
2021-09-26 21:29:12,524 - freqtrade.configuration.config_validation - INFO - Validating configuration ...
2021-09-26 21:29:12,526 - freqtrade.commands.optimize_commands - INFO - Starting freqtrade in Backtesting mode
2021-09-26 21:29:12,526 - freqtrade.exchange.exchange - INFO - Instance is running with dry_run enabled
2021-09-26 21:29:12,526 - freqtrade.exchange.exchange - INFO - Using CCXT 1.56.86
2021-09-26 21:29:12,526 - freqtrade.exchange.exchange - INFO - Applying additional ccxt config: {'enableRateLimit': True}
2021-09-26 21:29:12,529 - freqtrade.exchange.exchange - INFO - Applying additional ccxt config: {'enableRateLimit': True, 'rateLimit': 200}
2021-09-26 21:29:12,531 - freqtrade.exchange.exchange - INFO - Using Exchange "KuCoin"
2021-09-26 21:29:13,978 - freqtrade.resolvers.exchange_resolver - INFO - Using resolved exchange 'Kucoin'...
2021-09-26 21:29:13,997 - freqtrade.resolvers.iresolver - INFO - Using resolved strategy MyEnsembleStrategy from '/home/sage/ftworkdir/user_data/strategies/MyEnsembleStrategy.py'...
2021-09-26 21:29:13,997 - freqtrade.strategy.hyper - INFO - Found no parameter file.
2021-09-26 21:29:13,997 - freqtrade.strategy.hyper - INFO - No params for buy found, using default values.
2021-09-26 21:29:13,997 - freqtrade.strategy.hyper - INFO - No params for sell found, using default values.
2021-09-26 21:29:13,998 - freqtrade.strategy.hyper - INFO - No params for protection found, using default values.
2021-09-26 21:29:13,998 - MyEnsembleStrategy - INFO - Buy strategies: ['NotAnotherSMAOffsetStrategyModHO', 'Elliotv8', 'BigZ04_TSL4', 'NASOSv4']
2021-09-26 21:29:13,998 - freqtrade.resolvers.strategy_resolver - INFO - Override strategy 'timeframe' with value in config file: 5m.
2021-09-26 21:29:13,998 - freqtrade.resolvers.strategy_resolver - INFO - Override strategy 'order_types' with value in config file: {'buy': 'limit', 'sell': 'market', 'emergencysell': 'market', 'forcebuy': 'market', 'forcesell': 'market', 'stoploss': 'market', 'stoploss_on_exchange': False, 'stoploss_on_exchange_interval': 60}.
2021-09-26 21:29:13,998 - freqtrade.resolvers.strategy_resolver - INFO - Override strategy 'stake_currency' with value in config file: USDT.
2021-09-26 21:29:13,998 - freqtrade.resolvers.strategy_resolver - INFO - Override strategy 'stake_amount' with value in config file: unlimited.
2021-09-26 21:29:13,998 - freqtrade.resolvers.strategy_resolver - INFO - Override strategy 'unfilledtimeout' with value in config file: {'buy': 30, 'sell': 30, 'unit': '

In [51]:
get_backtest_repo().delete(0)

2021-09-28 16:19:28.268 | INFO     | lazyft.reports:delete:149 - Deleted report "c2c1aeb7-c597-4652-baf0-f49176cefb7f" from BacktestRepo


# Reports

## Hyperopt Reports

In [55]:
get_hyperopt_repo().head(25).df()

,strategy,date,hyperopt_id,exchange,m_o_t,stake,balance,score,avg_profit_pct,total_profit_pct,total_profit,trades,days,tag
0,MyEnsembleStrategy,09/27/21 16:28:26,15b05336-cd26-405f-bbc2-5e7f5f5b3e9a,kucoin,3,unlimited,100,14.208,4.440,300.670,300.670,96,30,"20210813-20210912,roi stoploss trailing"
1,MyEnsembleStrategy,09/27/21 09:15:48,dfd605a4-afb0-43f0-8a49-87bb67e8aeb2,kucoin,3,unlimited,100,14.324,4.430,305.350,305.350,97,30,"20210813-20210912,roi stoploss"
2,NASOSv5,09/27/21 07:43:22,73870aad-d55d-4dbf-b80a-1fa4ded3a7dd,kucoin,3,unlimited,100,3.872,3.630,317.790,317.790,128,120,"20210331-20210729,sell stoploss"
3,NASOSv5,09/27/21 05:57:07,18d342e5-44d0-42a9-90aa-b5dddc72571f,kucoin,3,unlimited,100,3.648,4.130,293.960,293.960,106,120,"20210331-20210729,buy roi"
4,Solipsis,09/26/21 16:03:30,c8df4b31-cb10-4c78-9bee-e7f65094bd0f,kucoin,3,unlimited,100,1.015,1.050,48.430,48.430,116,120,"20210330-20210728,sell stoploss"
5,Solipsis,09/25/21 20:51:57,b49f4fef-615a-4c2e-ac38-5581363f0651,kucoin,3,unlimited,100,0.665,0.790,30.020,30.020,101,120,20210329-20210727
6,BigZ04_TSL4,09/25/21 19:22:02,1d22c9d0-78c4-4daa-bdbc-ca590914a495,kucoin,3,unlimited,100,2.990,2.870,214.500,214.500,125,120,sell stoploss
7,BigZ04_TSL4,09/25/21 02:52:46,4fe9c951-43a3-4f92-8ffc-5a486141e646,kucoin,3,unlimited,100,2.381,2.070,147.790,147.790,138,120,20210328-20210726
8,NASOSv4,09/20/21 15:54:51,3db9cd99-1f65-4b2e-891f-c0544909e7ac,kucoin,3,unlimited,100,2.669,1.830,178.290,178.290,175,120,20210324-20210722
9,Elliotv8,09/20/21 12:02:12,03558ab6-cdc0-4ffa-930e-ab87d6e04c9a,kucoin,3,unlimited,100,2.607,2.370,173.310,173.310,132,120,20210324-20210722


In [4]:
get_hyperopt_repo()[2].parameters()

{
    'strategy_name': 'BigZ04_TSL4',
    'params': {
        'roi': {'0': 0.16699999999999998, '19': 0.049, '61': 0.023, '152': 0},
        'trailing': {'trailing_stop': False, 'trailing_stop_positive': 0.01, 'trailing_stop_positive_offset': 0.025, 'trailing_only_offset_is_reached': False},
        'buy': {
            'buy_bb20_close_bblowerband_safe_1': 0.991,
            'buy_bb20_close_bblowerband_safe_2': 0.97,
            'buy_condition_0_enable': True,
            'buy_condition_10_enable': True,
            'buy_condition_11_enable': False,
            'buy_condition_12_enable': True,
            'buy_condition_13_enable': True,
            'buy_condition_1_enable': False,
            'buy_condition_2_enable': False,
            'buy_condition_3_enable': False,
            'buy_condition_4_enable': True,
            'buy_condition_5_enable': True,
            'buy_condition_6_enable': True,
            'buy_condition_7_enable': True,
            'buy_condition_8_enable': True,
            'buy_condition_9_enable': True,
            'buy_dip_0': 1.018,
            'buy_macd_1': 0.08,
            'buy_macd_2': 0.05,
            'buy_rsi_0': 12.8,
            'buy_rsi_1': 18.9,
            'buy_rsi_1h_0': 75.0,
            'buy_rsi_1h_1': 14.8,
            'buy_rsi_1h_1a': 72.4,
            'buy_rsi_1h_2': 17.9,
            'buy_rsi_1h_3': 11.1,
            'buy_rsi_1h_4': 10.3,
            'buy_rsi_1h_5': 59.4,
            'buy_rsi_2': 25.8,
            'buy_rsi_3': 14.9,
            'buy_volume_drop_1': 4.1,
            'buy_volume_drop_2': 5.8,
            'buy_volume_drop_3': 4.9,
            'buy_volume_pump_1': 0.6
        },
        'sell': {'base_nb_candles_sell': 63, 'high_offset': 0.998, 'narrow_stop': 1.008, 'pHSL': -0.195, 'pPF_1': 0.009, 'pPF_2': 0.063, 'pSL_1': 0.011, 'pSL_2': 0.062, 'rsi_1h_val': 36, 'trade_time': 30, 'wide_stop': 1.045},
        'protection': {},
        'stoploss': {'stoploss': -0.288}
    },
    'ft_stratparam_v': 1,
    'export_time': '2021-09-25 23:21:37.827132+00:00'
}

In [36]:
get_hyperopt_repo().delete()

,strategy,date,hyperopt_id,exchange,m_o_t,stake,balance,score,avg_profit_pct,total_profit_pct,total_profit,trades,days,tag
0,MultiOffsetLamboV0,09/19/21 18:34:41,e42209a4-2c33-481b-baf5-5f9d96ded722,kucoin,3,unlimited,100,3.573,2.210,100.790,100.790,97,60,20210621-20210820
1,MultiOffsetLamboV0,09/19/21 17:43:51,3d05e06d-d499-4957-a7cc-561bce1e0b2d,kucoin,3,unlimited,100,4.125,2.230,124.200,124.200,111,60,20210621-20210820
2,NotAnotherSMAOffsetStrategyHOv3,09/19/21 15:10:44,273d711a-451b-466a-adee-7ed05a5ffe0e,kucoin,3,unlimited,100,5.966,1.140,219.700,219.700,314,60,20210621-20210820
3,CombinedBinHClucAndMADV9,09/18/21 22:03:25,60faed9e-7d81-4b55-9f86-32c131ed862b,kucoin,3,unlimited,100,1.991,1.890,368.380,368.380,256,243,20200918-20210519
4,Diamond,09/18/21 15:06:06,03875680-3612-4b7a-9cb0-ea9eca547b46,kucoin,3,unlimited,100,7.294,1.180,30K,30K,1502,243,20200918-20210519
5,MultiMA_TSL,09/17/21 17:27:04,4c7ab7ee-0a09-426c-9db5-0d6617262432,kucoin,3,unlimited,100,3.358,2.720,1.32K,1.32K,300,243,20200917-20210518
6,MultiMA_TSL,09/17/21 15:21:57,20d6ad94-c1c1-4a77-9737-f4e3414dd1fb,kucoin,3,unlimited,100,2.929,2.480,902.420,902.420,287,243,20200917-20210518
7,BigZ07,09/17/21 09:37:27,b648ca47-5f5b-4339-815f-ff6321c399d3,kucoin,3,unlimited,100,4.930,2.420,4.58K,4.58K,495,243,20200917-20210518
8,Elliotv8,09/16/21 23:48:39,b44bd6f7-06eb-4008-9676-bb2687ca7ddb,kucoin,3,unlimited,100,0.737,1.170,75.910,75.910,153,243,20200916-20210517
9,Elliotv8,09/16/21 23:44:02,8f4a32b0-7672-4c13-9558-f863c1c69008,kucoin,3,unlimited,100,1.210,1.710,154.490,154.490,172,243,20200916-20210517


## Backtest Reports

In [54]:
get_backtest_repo().head(20).df()

,strategy,date,hyperopt_id,exchange,m_o_t,stake,balance,score,avg_profit_pct,total_profit_pct,roiloss,winratioloss,sharpe_loss,sortino,total_profit,trades,days,tag
0,MyEnsembleStrategy,09/28/21 16:20:00,dfd605a4-afb0-43f0-8a49-87bb67e8aeb2,kucoin,3,unlimited,1000,1.201,1.802,0.012,-1.605,-360.313,-19.987,20.000,11.920,2,3,trailing-fix
1,MyEnsembleStrategy,09/27/21 08:28:33,,kucoin,3,unlimited,1000,4.497,2.237,333.575,-1.926,"-154,599.141",-16.614,-25.032,334K,808,402,trailing-fix
2,MyEnsembleStrategy,09/27/21 06:22:49,,kucoin,3,unlimited,1000,5.281,2.885,923.365,-2.495,"-172,782.794",-18.149,-27.362,923K,736,402,no modho
3,MyEnsembleStrategy,09/26/21 23:11:07,,kucoin,3,unlimited,1000,4.736,2.911,613.924,-2.477,"-169,706.336",-17.914,-26.251,614K,654,402,roi & stoploss
4,MyEnsembleStrategy,09/26/21 21:47:52,,kucoin,3,unlimited,1000,8.176,21.912,23.617,-18.297,"-291,428.076",-0.684,-22.331,23.6K,150,402,20200820-20210926
5,NASOSv5,09/26/21 18:46:33,,kucoin,5,unlimited,1000,15.110,1.149,1.720,-1.006,"-41,834.490",-51.122,-68.745,1.72K,447,34,20210820-20210926
6,MyEnsembleStrategy,09/26/21 17:44:36,,kucoin,5,unlimited,1000,10.632,3.347,1.036,-2.827,"-33,135.400",-63.296,-365.448,1.04K,108,34,TSL_fix
7,MyEnsembleStrategy,09/26/21 17:41:57,,kucoin,4,unlimited,1000,10.549,3.384,1.412,-2.858,"-32,820.635",-62.450,-359.439,1.41K,106,34,TSL_fix
8,MyEnsembleStrategy,09/26/21 17:27:25,,kucoin,3,unlimited,1000,10.436,3.412,2.184,-2.881,"-32,411.041",-61.333,-352.417,2.18K,104,34,TSL_fix
9,MyEnsembleStrategy,09/26/21 16:50:25,,kucoin,4,unlimited,1000,9.321,2.805,1.169,-2.375,"-28,607.875",-35.882,-56.379,1.17K,113,34,20210820-20210926


In [56]:
get_backtest_repo()[2].trades_to_csv()

2021-09-28 16:33:40.443 | INFO     | lazyft.models:trades_to_csv:292 - Created /home/sage/ftworkdir/exports/MyEnsembleStrategy-$1000-402_days.csv


In [36]:
get_backtest_repo().head(10).df()

,strategy,date,hyperopt_id,exchange,m_o_t,stake,balance,score,avg_profit_pct,total_profit_pct,roiloss,winratioloss,sharpe_loss,sortino,total_profit,trades,days,tag
0,NASOSv5,09/26/21 18:46:33,,kucoin,5,unlimited,1000,15.110,1.149,1.720,-1.006,"-41,834.490",-51.122,-68.745,1.72K,447,34,20210820-20210926
1,MyEnsembleStrategy,09/26/21 17:44:36,,kucoin,5,unlimited,1000,10.632,3.347,1.036,-2.827,"-33,135.400",-63.296,-365.448,1.04K,108,34,TSL_fix
2,MyEnsembleStrategy,09/26/21 17:41:57,,kucoin,4,unlimited,1000,10.549,3.384,1.412,-2.858,"-32,820.635",-62.450,-359.439,1.41K,106,34,TSL_fix
3,MyEnsembleStrategy,09/26/21 17:27:25,,kucoin,3,unlimited,1000,10.436,3.412,2.184,-2.881,"-32,411.041",-61.333,-352.417,2.18K,104,34,TSL_fix
4,MyEnsembleStrategy,09/26/21 16:50:25,,kucoin,4,unlimited,1000,9.321,2.805,1.169,-2.375,"-28,607.875",-35.882,-56.379,1.17K,113,34,20210820-20210926
5,MultiMA_TSL,09/26/21 16:47:05,4c7ab7ee-0a09-426c-9db5-0d6617262432,kucoin,3,unlimited,1000,4.010,1.894,0.554,-1.641,"-11,552.243",-13.836,-21.329,553.620,72,34,20210820-20210926
6,MyEnsembleStrategy,09/26/21 15:51:09,,kucoin,3,unlimited,1000,9.208,2.821,1.760,-2.388,"-28,206.082",-35.147,-55.229,1.76K,111,34,20210820-20210926
7,MyEnsembleStrategy2,09/25/21 20:44:55,,kucoin,3,unlimited,1000,4.398,1.335,0.609,-1.138,"-9,211.272",-14.746,-20.966,608.760,112,34,20210820-20210925
8,MyEnsembleStrategy,09/25/21 19:38:22,,kucoin,3,unlimited,1000,6.457,2.217,1.030,-1.884,"-19,069.894",-22.367,-31.753,1.03K,99,34,20210820-20210925
9,MyEnsembleStrategy,09/25/21 19:34:11,,kucoin,5,unlimited,1000,6.561,2.209,0.546,-1.877,"-19,436.129",-22.952,-32.567,546.320,101,34,20210820-20210925


In [3]:
get_backtest_repo()[0].trades_to_csv()

2021-09-27 01:22:09.009 | INFO     | lazyft.models:trades_to_csv:292 - Created /home/sage/ftworkdir/exports/MyEnsembleStrategy-$1000-402_days.csv


In [ ]:
get_backtest_repo().df().sort_values('roiloss').head(25)

,strategy,date,hyperopt_id,exchange,m_o_t,stake,balance,score,avg_profit_pct,total_profit_pct,roiloss,winratioloss,sharpe_loss,sortino,total_profit,trades,days,tag
67,NASOSv4,08/30/21 17:54:17,,kucoin,3,unlimited,100,11.26,111.06,15.20,-92.57,"-77,741.51",-0.72,-33.82,1.52K,37,365,
14,NASOSv4,09/15/21 16:09:24,,binanceus,3,unlimited,100,4.08,17.68,0.28,-14.75,"-5,303.01",-1.63,-10.25,27.51,6,26,20210820-
5,NASOSv4,09/17/21 15:43:53,3da94866-ccb5-4821-af61-b69d687b8d04,kucoin,3,unlimited,100,6.19,6.42,0.72,-5.51,"-14,130.86",-28.40,20.00,71.81,26,27,20210820-20210917
2,NASOSv4,09/17/21 15:56:37,3da94866-ccb5-4821-af61-b69d687b8d04,kucoin,3,unlimited,100,2.77,4.81,28.99,-4.16,"-87,495.62",-7.07,-10.25,2.9K,221,383,20200820-20210917
4,NotAnotherSMAOffsetStrategyModHO,09/17/21 15:44:31,94db2236-0861-4d7b-b720-7ecd04999290,kucoin,3,unlimited,100,6.48,3.97,0.75,-3.44,"-13,911.04",-15.92,-35.41,74.71,44,27,20210820-20210917
13,NotAnotherSMAOffsetStrategyModHO,09/16/21 00:13:51,94db2236-0861-4d7b-b720-7ecd04999290,kucoin,3,unlimited,100,6.48,3.97,0.75,-3.44,"-13,911.04",-15.92,-35.41,74.71,44,27,20210820-20210916
12,NASOSv4,09/16/21 00:14:14,,kucoin,3,unlimited,100,1.06,4.08,0.10,-3.42,"-1,225.24",-0.89,-2.66,9.70,7,27,20210820-20210916
1,NotAnotherSMAOffsetStrategyModHO,09/17/21 15:58:28,94db2236-0861-4d7b-b720-7ecd04999290,kucoin,3,unlimited,100,3.50,3.76,64.31,-3.26,"-98,975.53",-7.84,-17.92,6.43K,356,383,20200820-20210917
79,NotAnotherSMAOffsetStrategyModHO,08/28/21 14:50:06,94db2236-0861-4d7b-b720-7ecd04999290,kucoin,4,unlimited,1400,3.87,3.53,27.03,-3.06,"-104,370.02",-8.65,-19.16,37.8K,401,365,
85,NotAnotherSMAOffsetStrategyModHO,08/27/21 00:59:03,94db2236-0861-4d7b-b720-7ecd04999290,kucoin,4,unlimited,1400,3.68,2.64,0.28,-2.33,"-7,925.06",-8.34,-15.40,390.30,39,28,


In [37]:
get_backtest_repo().filter_by_strategy('MyEnsembleStrategy').get_pair_totals().sort_values('profit_sum_pct', ascending=False).tail(15).index.tolist()

['CAKE/USDT', 'LINK/USDT', 'BCH/USDT', 'ERG/USDT', 'MANA/USDT', 'XDC/USDT', 'NEO/USDT', 'ENJ/USDT', 'ICP/USDT', 'CHZ/USDT', 'SAND/USDT', 'SXP/USDT', 'ONE/USDT', 'CQT/USDT', 'SUSHI/USDT']

In [22]:
(get_backtest_repo()
.df()
 .query('days==30')
 .sort_values(by=['avg_profit_pct'], ascending=False))

,strategy,date,hyperopt_id,exchange,m_o_t,stake,balance,score,avg_profit_pct,total_profit_pct,roiloss,winratioloss,sharpe_loss,sortino,total_profit,trades,days,tag
2,MyEnsembleStrategy,09/20/21 13:16:57,,kucoin,5,unlimited,1000,10.857,2.157,0.893,-1.830,"-25,022.165",-44.752,-86.197,893.360,151,30,mot5_3
6,MyEnsembleStrategy,09/20/21 12:05:18,,kucoin,3,unlimited,100,9.314,1.914,1.459,-1.625,"-20,094.724",-35.762,-68.341,145.890,146,30,elliot-reopted
11,MyEnsembleStrategy,09/19/21 20:38:53,,kucoin,3,unlimited,100,8.800,1.899,1.337,-1.612,"-18,802.400",-33.475,-64.157,133.720,139,30,RalliV1-opted
4,MyEnsembleStrategy,09/20/21 13:03:11,,kucoin,5,unlimited,1000,10.937,1.897,0.902,-1.613,"-25,224.374",-47.572,-90.485,901.940,173,30,mot5
12,MultiOffsetLamboV0,09/19/21 17:46:22,3d05e06d-d499-4957-a7cc-561bce1e0b2d,kucoin,3,unlimited,100,7.863,1.872,1.145,-1.611,"-18,720.761",-31.605,-49.542,114.500,126,30,MultiOffsetLamboV0-opted
10,MyEnsembleStrategy,09/19/21 23:22:18,,kucoin,3,unlimited,100,8.443,1.862,1.382,-1.584,"-19,555.516",-29.139,-42.116,138.250,136,30,NASMAOHOv3-opted
0,MyEnsembleStrategy,09/20/21 14:21:20,,kucoin,5,unlimited,1000,12.719,1.826,1.110,-1.552,"-28,116.491",-52.338,-99.226,1.11K,209,30,mot5_5
5,MyEnsembleStrategy,09/20/21 12:54:12,,kucoin,3,unlimited,100,10.188,1.819,1.699,-1.548,"-23,468.702",-43.921,-80.068,169.880,168,30,roifix
7,MyEnsembleStrategy,09/20/21 03:57:47,,kucoin,3,unlimited,100,9.553,1.814,1.511,-1.542,"-20,859.316",-35.924,-61.918,151.070,158,30,trimmed
18,MyEnsembleStrategy,09/19/21 14:16:32,,kucoin,3,unlimited,100,8.995,1.329,1.372,-1.137,"-19,008.157",-36.127,-62.830,137.240,203,30,20210820-20210919


In [23]:
index = 90
get_backtest_repo()[index].pair_performance
get_backtest_repo()[index].sell_reason_summary
get_backtest_repo()[index].as_df('left_open_trades')
get_backtest_repo()[0].backtest_data.keys()

,key,trades,profit_mean,profit_mean_pct,profit_sum,profit_sum_pct,profit_total_abs,profit_total,profit_total_pct,duration_avg,wins,draws,losses
0,ARX/USDT,68,0.037,3.661,2.489,248.930,944.144,9.441,944.140,0:38:00,59,0,9
1,FTM/USDT,4,0.090,8.999,0.360,36.000,475.834,4.758,475.830,0:11:00,4,0,0
2,QI/USDT,13,0.019,1.893,0.246,24.610,294.873,2.949,294.870,0:37:00,12,0,1
3,VRA/USDT,35,0.028,2.792,0.977,97.710,257.485,2.575,257.480,0:43:00,30,0,5
4,LABS/USDT,11,0.018,1.773,0.195,19.510,234.684,2.347,234.680,0:55:00,9,0,2
5,DAG/USDT,16,0.020,1.994,0.319,31.910,228.997,2.290,229.000,1:02:00,12,0,4
6,XAVA/USDT,8,0.023,2.350,0.188,18.800,179.126,1.791,179.130,1:09:00,6,0,2
7,VET/USDT,6,0.077,7.681,0.461,46.090,170.400,1.704,170.400,0:13:00,6,0,0
8,HTR/USDT,33,0.025,2.514,0.829,82.950,160.112,1.601,160.110,0:23:00,30,0,3
9,SHA/USDT,23,0.018,1.796,0.413,41.300,142.360,1.424,142.360,0:49:00,21,0,2


,sell_reason,trades,wins,draws,losses,profit_mean,profit_mean_pct,profit_sum,profit_sum_pct,profit_total_abs,profit_total,profit_total_pct
0,New Sell Signal (trima zema hma ),152,137,0,15,0.037,3.700,5.620,561.980,2.01K,1.873,187.330
1,New Sell Signal (trima ),140,126,0,14,0.028,2.840,3.981,398.100,1.28K,1.327,132.700
2,New Sell Signal (trima zema ),64,57,0,7,0.039,3.930,2.514,251.430,1.08K,0.838,83.810
3,New Sell Signal (zema ),43,35,0,8,0.017,1.730,0.744,74.450,301.518,0.248,24.820
4,trailing_stop_loss,33,33,0,0,0.040,4.010,1.324,132.360,800.467,0.441,44.120
5,stop_loss,21,0,0,21,-0.152,-15.170,-3.186,-318.570,"-1,614.833",-1.062,-106.190
6,New Sell Signal (zema hma ),14,14,0,0,0.014,1.360,0.191,19.090,116.906,0.064,6.360
7,New Sell Signal (trima hma ),7,7,0,0,0.061,6.060,0.424,42.450,175.182,0.141,14.150
8,New Sell Signal (hma ),1,1,0,0,0.051,5.140,0.051,5.140,49.599,0.017,1.710


,key,trades,profit_mean,profit_mean_pct,profit_sum,profit_sum_pct,profit_total_abs,profit_total,profit_total_pct,duration_avg,wins,draws,losses
0,TOTAL,0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0:00,0,0,0


dict_keys(['trades', 'locks', 'best_pair', 'worst_pair', 'results_per_pair', 'sell_reason_summary', 'left_open_trades', 'total_trades', 'total_volume', 'avg_stake_amount', 'profit_mean', 'profit_median', 'profit_total', 'profit_total_abs', 'backtest_start', 'backtest_start_ts', 'backtest_end', 'backtest_end_ts', 'backtest_days', 'backtest_run_start_ts', 'backtest_run_end_ts', 'trades_per_day', 'market_change', 'pairlist', 'stake_amount', 'stake_currency', 'stake_currency_decimals', 'starting_balance', 'dry_run_wallet', 'final_balance', 'rejected_signals', 'max_open_trades', 'max_open_trades_setting', 'timeframe', 'timeframe_detail', 'timerange', 'enable_protections', 'strategy_name', 'stoploss', 'trailing_stop', 'trailing_stop_positive', 'trailing_stop_positive_offset', 'trailing_only_offset_is_reached', 'use_custom_stoploss', 'minimal_roi', 'use_sell_signal', 'sell_profit_only', 'sell_profit_offset', 'ignore_roi_if_buy_signal', 'backtest_best_day', 'backtest_worst_day', 'backtest_best_day_abs', 'backtest_worst_day_abs', 'winning_days', 'draw_days', 'losing_days', 'daily_profit', 'wins', 'losses', 'draws', 'holding_avg', 'holding_avg_s', 'winner_holding_avg', 'winner_holding_avg_s', 'loser_holding_avg', 'loser_holding_avg_s', 'max_drawdown', 'max_drawdown_abs', 'drawdown_start', 'drawdown_start_ts', 'drawdown_end', 'drawdown_end_ts', 'max_drawdown_low', 'max_drawdown_high', 'csum_min', 'csum_max'])

In [ ]:
# import rapidjson

# rapidjson.loads(
#     ParameterTools.get_path_of_params(
#         "ca1f7c9b-e3d6-483b-b912-00688410f0ed"
#     ).read_text()
# )
get_hyperopt_repo().get_by_param_id('b40294bf-de3b-4f7d-a6f7-a412709e549c').parameters()

# Remote

In [23]:
bot1 = remote.RemoteBot(1, "vps")
bot4 = remote.RemoteBot(4, "vps")
bot2 = remote.RemoteBot(2, "pi")
bot3 = remote.RemoteBot(3, "pi")
bot5 = remote.RemoteBot(5, "pi")

In [ ]:
bot4.tools.send_file(4, str(priv_us), "user_data/priv_us.json")

In [ ]:
c = remote.RemoteBotInfo(1).config
c.pairslist

In [ ]:
bot4.update_config(dict(max_open_trades=3))
bot4.restart()

In [53]:
# bot = bot4
# for s in b_params.ensemble:
#     bot.set_strategy(s.name, id=s.id)
bot.set_strategy('MyEnsembleStrategy', id='dfd605a4-afb0-43f0-8a49-87bb67e8aeb2')
# bot.set_run_mode("live")
bot.env
# bot.update_ensemble(b_params.ensemble)
bot.restart()

2021-09-28 16:24:34.889 | INFO     | lazyft.strategy:create_strategy_params_filepath:97 - Getting parameters file of MyEnsembleStrategy


Environment(data={'STRATEGY': 'MyEnsembleStrategy', 'ID': 'dfd605a4-afb0-43f0-8a49-87bb67e8aeb2', 'DB_FILE': 'tradesv3.MyEnsembleStrategy-dry-dfd605a4-afb0-43f0-8a49-87bb67e8aeb2.sqlite'}, file=PosixPath('/tmp/lazyftczs9qs1n/.env'))

2021-09-28 16:24:40.984 | INFO     | lazyft.remote:<lambda>:326 - Stopping freqtrade_bot4 ...
2021-09-28 16:24:51.495 | INFO     | lazyft.remote:<lambda>:326 - Stopping freqtrade_bot4 ... done
2021-09-28 16:24:51.530 | INFO     | lazyft.remote:<lambda>:326 - Removing freqtrade_bot4 ...
2021-09-28 16:24:51.604 | INFO     | lazyft.remote:<lambda>:326 - Removing freqtrade_bot4 ... done
2021-09-28 16:24:51.607 | INFO     | lazyft.remote:<lambda>:326 - Removing network freqtrade-bot4_default
2021-09-28 16:24:52.558 | INFO     | lazyft.remote:<lambda>:326 - Creating network "freqtrade-bot4_default" with the default driver
2021-09-28 16:24:52.698 | INFO     | lazyft.remote:<lambda>:326 - Creating freqtrade_bot4 ...
2021-09-28 16:24:53.798 | INFO     | lazyft.remote:<lambda>:326 - Creating freqtrade_bot4 ... done


In [34]:
bot4.get_trades()

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /home/sage/freqtrade/freqtrade/persistence/models.py:53 in init_db                               │
│                                                                                                  │
│    50 │   │   })                                                                                 │
│    51 │                                                                                          │
│    52 │   try:                                                                                   │
│ ❱  53 │   │   engine = create_engine(db_url, future=True, **kwargs)                              │
│    54 │   except NoSuchModuleError:                                                              │
│    55 │   │   raise OperationalException(f"Given value for db_url: '{db_url}' "                  │
│    56 │   │   │   │   │   │   │   │      f"is no valid database URL! (See {_SQL_DOCS_URL})")     │
│                                                                                                  │
│ ╭────────────────────────────────────────── locals ──────────────────────────────────────────╮   │
│ │ clean_open_orders = False                                                                  │   │
│ │            db_url = 'sqlite:////tmp/lazyft5pre8osd/tradesv3.MyEnsembleStrategy-dry.sqlite' │   │
│ │            kwargs = {'connect_args': {'check_same_thread': False}}                         │   │
│ ╰────────────────────────────────────────────────────────────────────────────────────────────╯   │
│ <string>:2 in create_engine                                                                      │
│ ╭──────────────────────────────────── locals ─────────────────────────────────────╮              │
│ │ kwargs = {'future': True, 'connect_args': {'check_same_thread': False}}         │              │
│ │    url = 'sqlite:////tmp/lazyft5pre8osd/tradesv3.MyEnsembleStrategy-dry.sqlite' │              │
│ ╰─────────────────────────────────────────────────────────────────────────────────╯              │
│                                                                                                  │
│ /home/sage/ftworkdir/.direnv/python-3.9.6/lib/python3.9/site-packages/sqlalchemy/util/deprecatio │
│                                                                                                  │
│   295 │   │   │   │   │   │   version_warnings[m],                                               │
│   296 │   │   │   │   │   │   stacklevel=3,                                                      │
│   297 │   │   │   │   │   )                                                                      │
│ ❱ 298 │   │   │   return fn(*args, **kwargs)                                                     │
│   299 │   │                                                                                      │
│   300 │   │   doc = fn.__doc__ is not None and fn.__doc__ or ""                                  │
│   301 │   │   if doc:                                                                            │
│                                                                                                  │
│ ╭─────────────────────────────────────────── locals ───────────────────────────────────────────╮ │
│ │             args = ('sqlite:////tmp/lazyft5pre8osd/tradesv3.MyEnsembleStrategy-dry.sqlite',) │ │
│ │     check_any_kw = 'kwargs'                                                                  │ │
│ │   check_defaults = ()                                                                        │ │
│ │         check_kw = {'empty_in_strategy', 'strategy', 'case_sensitive'}                       │ │
│ │               fn = <function create_engine at 0x7fd5e1139820>                                │ │
│ │           kwargs = {'future': True, 'connect_args': {'check_same_thread': False}}            │ │
│ │                m = 'case_sensitive'                      

In [ ]:
bot2.get_trades().profit_abs.sum()

In [51]:
from tempfile import TemporaryDirectory
from pathlib import Path
def update_ensemble(self, strategies: list[Strategy]):
        tmp_path = TemporaryDirectory()
        path = Path(tmp_path.name, 'ensemble.json')
        path.write_text(rapidjson.dumps([s.name for s in strategies]))
        self.tools.send_file(
            self.bot_id, path, 'user_data/strategies/ensemble.json'
        )
remote.RemoteBot.update_ensemble=update_ensemble

In [52]:
# bot3.restart()
# bot5.restart()
import rapidjson
bot.update_ensemble(b_params.ensemble)

# Misc

In [29]:
!freqtrade hyperopt-show --best

2021-09-27 16:27:48,385 - freqtrade.loggers - INFO - Verbosity set to 0
2021-09-27 16:27:48,385 - freqtrade.configuration.configuration - INFO - Using user-data directory: /home/sage/ftworkdir/user_data ...
2021-09-27 16:27:48,385 - freqtrade.configuration.configuration - INFO - Using data directory: /home/sage/ftworkdir/user_data/data ...
2021-09-27 16:27:48,385 - freqtrade.configuration.configuration - INFO - Parameter --best detected: True
2021-09-27 16:27:48,385 - freqtrade.configuration.config_validation - INFO - Validating configuration ...
2021-09-27 16:27:48,386 - freqtrade.optimize.hyperopt_tools - INFO - Reading epochs from '/home/sage/ftworkdir/user_data/hyperopt_results/strategy_MyEnsembleStrategy_2021-09-27_09-16-50.fthypt'
2021-09-27 16:27:49,222 - freqtrade.optimize.hyperopt_tools - INFO - Loaded 750 previous evaluations from disk.
2021-09-27 16:27:49,222 - freqtrade.optimize.hyperopt_epoch_filters - INFO - 21 best epochs found.
Result for strategy MyEnsembleStrategy
===

In [34]:
!freqtrade hyperopt --best

usage: freqtrade [-h] [-V]
                 {trade,create-userdir,new-config,new-strategy,download-data,convert-data,convert-trade-data,list-data,backtesting,edge,hyperopt,hyperopt-list,hyperopt-show,list-exchanges,list-markets,list-pairs,list-strategies,list-timeframes,show-trades,test-pairlist,install-ui,plot-dataframe,plot-profit,webserver}
                 ...
freqtrade: error: unrecognized arguments: --best


In [33]:
!freqtrade list-strategies

2021-09-26 20:05:33,880 - freqtrade.loggers - INFO - Verbosity set to 0
2021-09-26 20:05:33,880 - freqtrade.configuration.configuration - INFO - Using user-data directory: /home/sage/ftworkdir/user_data ...
2021-09-26 20:05:33,881 - freqtrade.configuration.configuration - INFO - Using data directory: /home/sage/ftworkdir/user_data/data ...
2021-09-26 20:05:33,881 - freqtrade.configuration.config_validation - INFO - Validating configuration ...
2021-09-26 20:05:33,955 - freqtrade.resolvers.iresolver - WARNING - Could not import /home/sage/ftworkdir/user_data/strategies/test_binh.py due to 'No module named 'custom_util''
+-------------------------------------------+----------------------------------------------+-------------+----------------+--------------+---------------+
|                                      name |                                     location |      status |   hyperoptable |   buy-Params |   sell-Params |
|-------------------------------------------+------------------

In [ ]:
!echo "hi"

In [ ]:
!tail --help

In [4]:
print('hi')

hi

In [12]:
!pip install finta